## *Submitted by:* `Sunta Pradhan`

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

In [3]:
# Define transformations to apply to the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:02<00:00, 78658921.47it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Define dataloaders for training and testing
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

In [6]:
# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Define a simple CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
# Instantiate the CNN models
models = []
models.append(Net().to(device))
models.append(Net().to(device))
models.append(Net().to(device))
models.append(Net().to(device))
models.append(Net().to(device))

In [9]:
# Define the loss function and optimizer for each model
criterion = nn.CrossEntropyLoss()
optimizers = [optim.Adam(model.parameters(), lr=0.001) for model in models]

In [10]:
# Training loop for each model
for model, optimizer in zip(models, optimizers):
    for epoch in range(10):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            # Resize inputs to match target labels
            inputs = torch.nn.functional.interpolate(inputs, size=(32, 32))

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"[Model {models.index(model) + 1}] [{epoch + 1}] Loss: {running_loss / len(trainloader)}")

[Model 1] [1] Loss: 1.4864670788235677
[Model 1] [2] Loss: 1.1259023027346873
[Model 1] [3] Loss: 0.9687344375473764
[Model 1] [4] Loss: 0.8816523660174416
[Model 1] [5] Loss: 0.8000915047457761
[Model 1] [6] Loss: 0.7324154758849717
[Model 1] [7] Loss: 0.681418021469165
[Model 1] [8] Loss: 0.6314457207537063
[Model 1] [9] Loss: 0.5836518460222523
[Model 1] [10] Loss: 0.5405042504563051
[Model 2] [1] Loss: 1.4399494734566536
[Model 2] [2] Loss: 1.0624853579894356
[Model 2] [3] Loss: 0.9096878802075106
[Model 2] [4] Loss: 0.8188221036930523
[Model 2] [5] Loss: 0.7436945726499533
[Model 2] [6] Loss: 0.6773942728786517
[Model 2] [7] Loss: 0.6187874684705759
[Model 2] [8] Loss: 0.5638641464283399
[Model 2] [9] Loss: 0.5140456307269728
[Model 2] [10] Loss: 0.4633843502425172
[Model 3] [1] Loss: 1.4699561129445615
[Model 3] [2] Loss: 1.0933105422712652
[Model 3] [3] Loss: 0.9404634345522926
[Model 3] [4] Loss: 0.8508815346166606
[Model 3] [5] Loss: 0.7820139268170232
[Model 3] [6] Loss: 0.72

In [11]:
# Evaluate each model on the test set
accuracies = []
for model in models:
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            
            # Resize inputs to match target labels
            images = torch.nn.functional.interpolate(images, size=(32, 32))

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    accuracies.append(accuracy)
    print(f"[Model {models.index(model) + 1}] Validation Accuracy: {accuracy:.2f}%")

[Model 1] Validation Accuracy: 72.24%
[Model 2] Validation Accuracy: 72.39%
[Model 3] Validation Accuracy: 71.84%
[Model 4] Validation Accuracy: 72.82%
[Model 5] Validation Accuracy: 71.86%


### Thank you..😊